# Merge all auctions into one

Combines several datasets and do some simple analyses.  


### User variables

In [1]:
import os
# setting path
os.chdir(r'..')

import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
SKIPSAVE = cfg['SKIPSAVE']

TAG_SINGLE = "nbconvert_instruction:remove_single_output"

if VERBOSE > 0:
    display(cfg)

{'settings_fn': '../code/assets/drz-auction-settings.ini',
 'DATE': '2023-10',
 'VERBOSE': 1,
 'OPBOD': False,
 'URL': 'http://verkoop.domeinenrz.nl/verkoop_bij_inschrijving_2023-0010',
 'EXTEND_URL': False,
 'CLOSEDDATA': True,
 'closed_data_fields': '*',
 'SKIPSAVE': False}

In [2]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
if OPBOD:
    AuctionDates = ['2019-11', '2019-12', 
                    '2020-01', '2020-02', '2020-03', '2021-09-09', '2021-10-10', '2021-11-11', '2021-12-12', 
                    '2022-01-01', '2022-02-02', '2022-04-04', '2022-05-05', '2022-06-06', '2022-08-08', '2022-09-09', '2022-10-10', '2022-11-11', '2022-12-12',
                    '2023-01-01', '2023-02-02', '2023-03-03', '2023-04-04', '2023-05-05'
                   ]
else:    
    AuctionDates = ['2014-10', '2014-11', '2014-12', # old!
                    '2015-01', '2015-02', '2015-03', # old!
                    '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
                    '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                    '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                    '2020-01', '2020-02', '2020-03', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                    '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', 
                    '2021-06', '2021-06-16', '2021-07-07',  '2021-07-17', '2021-08-08', '2021-08-18', '2021-09-09', '2021-09-19', '2021-10-10', '2021-10-20', '2021-11-11', '2021-11-21', '2021-12-12', '2021-12-22',
                    '2022-01-01', '2022-01-21', '2022-02-02', '2022-02-22', '2022-03-03', '2022-03-23', '2022-04-04', '2022-04-24', '2022-05-05', '2022-05-25', '2022-06-06', 
                    '2022-07-07', '2022-07-27', '2022-08-08', '2022-08-28', '2022-09-09', '2022-09-29', '2022-10-10', '2022-10-30', '2022-11-11', '2022-11-31', '2022-12-12', '2022-12-32',
                    '2023-01-01', '2023-02-02', '2023-03-03', '2023-04-04', '2023-05-05', '2023-06-06', '2023-07-07', '2023-08-08', '2023-09-09', '2023-10-10'
        
                   ]  
if DATE not in ['-'.join(d.split('-')[:2]) for d in AuctionDates]:
    raise ValueError(f'{DATE} not in list that will be concatenated. Add to list.')

### Import modules 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
plt.style.use('ggplot')

### Load reference tables

In [4]:
name_table = pd.read_excel('./assets/fieldname-lookuptable.ods', sheet_name='preferred_name', header=0, skipfooter=1, dtype=str)
name_table = name_table.dropna(how='all', axis=0).dropna(how='all', axis=1)
rename_dict={}
for i,r in name_table.iterrows():
    r=r.dropna()
    k = r.iloc[0]
    for option in r.iloc[1:]:
        rename_dict[option] = k

print(len(rename_dict))

535


In [5]:
# some have trailing _X-s as counter
# if option does not have counter use _1
pat = '(_X)+$'
for from_name, to_name in rename_dict.items():
    # count nr of _X
    lvl = [None,None]
    M = re.search(pat, from_name); 
    lvl[0] = int((M.end() - M.start())/2) if M is not None else 0
    M = re.search(pat, to_name); 
    lvl[1] = int((M.end() - M.start())/2) if M is not None else 0

    # if disbalance assume _1 
    # rename last _X to _1 repeat for diff in _X
    if lvl[0]<lvl[1]:
        for i in range(lvl[1]-lvl[0]):
            to_name = re.sub('_X(_1)?$', '_1\g<1>', to_name)
        rename_dict[from_name] = to_name

In [6]:
rename_dict_with_counter = {}
for k,v in rename_dict.items():
    pat = '_X(_\d)?$'
    if re.search(pat, v) is not None:
        # should have equal nr of _X
        n = [None,None]
        M = re.search(pat, k)
        n[0] = len(re.findall('_X', M[0]))
        M = re.search(pat, v)
        n[1] = len(re.findall('_X', M[0]))
        assert n[0] == n[1]
        rename_dict_with_counter[k] = v

for k in rename_dict_with_counter.keys():
    del rename_dict[k]

print(len(rename_dict), len(rename_dict_with_counter))

457 78


### Load all data

Auctions are saved monthly in a .pkl file

In [7]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    if OPBOD:
        fn = '../data/auctions/enriched-results/rdw-data-{:s}-opbod.pkl'.format(
            AuctionDate
        )

    else:    
        fn = '../data/auctions/enriched-results/rdw-data-{:s}.pkl'.format(
            AuctionDate
        )

    if VERBOSE > 0: print(fn)
    df = pd.read_pickle(fn)
    
    # flatten column levels
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.map(lambda x: '_'.join(x))# if x[0] != 'drz' else '_'.join(x[1:]))
        
    # store in dict
    data[AuctionDate] = df
    
    
# When there are two auctions in a single month
if not OPBOD:
    # HACK ALERT!!
    data['2021-06-16'].index = data['2021-06-16'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    data['2021-06'].index = data['2021-06'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    
    # matrix of auction date
    auct_month = np.array([[int(k.split('-')[0]), int(k.split('-')[1])] for k in data.keys()])
    auct_month = np.unique(auct_month, axis=0)

    # 2021-07 there is a systematic index change
    for m in auct_month[(auct_month[:,0] == 2021) & (auct_month[:,1] >= 7) | (auct_month[:,0] >= 2022)]:
        for c in range(2):

            # multiplier
            #   2021: 10
            #   2022, 2023: 20
            if m[0] == 2021:
                multiplier = 10
            elif m[0] >= 2022:
                multiplier = 20
            else:
                raise NotImplementedError

            # data key
            k = f'{m[0]}-{m[1]:02.0f}-{m[1]+(c*multiplier):02.0f}'
            if k not in data.keys():
                # skip if not exist
                continue

            # translation
            pat = '{0}-' + f'{m[1]:02.0f}' + '-{2}{1}'
            data[k].index = data[k].index.map(lambda x: pat.format(*x.split('-')))

            if VERBOSE > 0:
                print(f'{k} > {data[k].index[0]} .. {data[k].index[-1]}')
if VERBOSE > 0:
    display({'text/plain': '\nlast rows of last file'}, raw=True, metadata={"tags":(TAG_SINGLE, )})
    display(data[AuctionDates[-1]].tail(), metadata={"tags":(TAG_SINGLE, )})

load data
../data/auctions/enriched-results/rdw-data-2014-10.pkl
../data/auctions/enriched-results/rdw-data-2014-11.pkl
../data/auctions/enriched-results/rdw-data-2014-12.pkl
../data/auctions/enriched-results/rdw-data-2015-01.pkl
../data/auctions/enriched-results/rdw-data-2015-02.pkl
../data/auctions/enriched-results/rdw-data-2015-03.pkl
../data/auctions/enriched-results/rdw-data-2017-03.pkl
../data/auctions/enriched-results/rdw-data-2017-04.pkl
../data/auctions/enriched-results/rdw-data-2017-05.pkl
../data/auctions/enriched-results/rdw-data-2017-06.pkl
../data/auctions/enriched-results/rdw-data-2017-07.pkl
../data/auctions/enriched-results/rdw-data-2017-08.pkl
../data/auctions/enriched-results/rdw-data-2017-09.pkl
../data/auctions/enriched-results/rdw-data-2017-10.pkl
../data/auctions/enriched-results/rdw-data-2017-11.pkl
../data/auctions/enriched-results/rdw-data-2017-12.pkl
../data/auctions/enriched-results/rdw-data-2018-01.pkl
../data/auctions/enriched-results/rdw-data-2018-02.pkl


### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [8]:
# Change type of early auctions for images
if VERBOSE > 0:
    print('Fields with images look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first row'])
    for k in data:
        im_cols = data[k].columns[
            data[k].columns.str.lower().isin(['image', 'images', 'drz__images'])
        ]
        example = data[k][im_cols[0]].dropna().values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = im_cols[0]
        if isinstance(example, str):
            df_.loc[k, 'first row'] = example
        else:
            df_.loc[k, 'first row'] = len(example)
    with pd.option_context('display.max_row', 999):
        display(df_)

if OPBOD:
    # reformatting did not happen in IRS auctions
    pass
else:
    
    # convert string representation of list to real list
    # "['image1', 'image2']"
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        df_ = data[k].copy() # avoid performance warning
        df_['drz__Images'] = df_['Image'].apply(eval)
        data[k] = df_.drop(columns=['Image'])

    # convert string representation of list without quotes to real list
    # "[image1, image2]"
    for k in ['2017-11',
              '2017-12',
              '2018-01',
              '2018-02',
              '2018-03',
              '2018-04']:
        df_ = data[k].copy()
        df_['drz__Images'] = df_['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
        data[k] = df_.drop(columns=['Images'])
    

Fields with images look like this:



,column,type,first row
2014-10,Images,<class 'list'>,0
2014-11,Images,<class 'list'>,0
2014-12,Images,<class 'list'>,0
2015-01,Images,<class 'list'>,0
2015-02,Images,<class 'list'>,0
2015-03,Images,<class 'list'>,0
2017-03,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14883593...
2017-04,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14906980...
2017-05,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14931056...
2017-06,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14962307...


In [9]:
VERBOSE = 1
# Change dicttype of early auctions to regular fields
if VERBOSE > 0:
    print('rdwinfo look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first length'])
    for k in data:
        ri_cols = data[k].columns[
            data[k].columns.str.lower().isin(['rdwinfo'])
        ]
        if ri_cols.shape[0] == 0:
            continue
        example = data[k][ri_cols[0]].dropna().values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = ri_cols[0]
        df_.loc[k, 'first length'] = len(example)
    with pd.option_context('display.max_row', 999):
        display(df_.dropna(how='all'))

if OPBOD:
    NotImplementedError
else:
    

    # Convert list to dict first
    # df_.groupby('type').get_group("<class 'list'>").index
    for k in ['2017-04',
              '2017-05',
              '2018-07',
              '2018-09',
              '2018-10']:
        df_ = data[k].copy()
        assert all(df_['rdwinfo'].apply(lambda x: len(x) if isinstance(x, list) else 0)<=1), 'Which item in list shoud i pick'
        df_['rdwinfo'] = df_['rdwinfo'].apply(lambda x: x[0] if isinstance(x, list) else None)
        data[k] = df_

    # expand dict to DF columns
    # df_.groupby('type').get_group("<class 'dict'>").index
    for k in [
        '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
        '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
        '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
        '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
        '2019-03'
    ]:
        df_ = data[k].copy()
        columns = set(df_['rdwinfo'].apply(lambda x: list(x.keys()) if x is not None else []).sum())

        df_ri = pd.json_normalize(df_['rdwinfo'], sep='|')
        df_ri.index = df_.index
        df_ri.columns = df_ri.columns.str.split('|', expand=True)
        columns_with_suffix = df_ri.loc[:, (slice(None), '1')].columns.get_level_values(0)
        columns_wo_suffix = np.setdiff1d(df_ri.columns.get_level_values(0), columns_with_suffix)

        assert df_ri.columns.nlevels == 2, 'more splits in columns'
        new_col_names = dict()
        for c in df_ri.columns:
            if c[0] in columns_wo_suffix:
                new_col_names[c] = c[0]
            elif c[0] in columns_with_suffix:
                new_col_names[c] = c[0] + '_' + str(int(c[1])+1)
        df_ri.columns = new_col_names.values()
        df_ri = df_ri.add_prefix('rdw_')
        # kenteken_1 and _2 in set. reduce to one
        cols = [c for c in df_ri.columns if c.startswith('rdw_kenteken_')]
        if len(cols) > 0:
            reg_plate = df_ri.loc[:, cols].bfill(axis=1).ffill(axis=1)
            assert (reg_plate.fillna('').nunique(axis=1) > 1).any() == False, 'regs not equal'
            df_ri['rdw_kenteken'] = reg_plate.rdw_kenteken_1
            df_ri.drop(columns=cols, inplace=True)
        existing_cols = np.intersect1d(df_ri.columns, df_.columns).tolist()
        existing_cols += ['rdwinfo']
        data[k] = pd.merge(df_.drop(columns=existing_cols), df_ri, left_index=True, right_index=True)


rdwinfo look like this:



,column,type,first length
2017-03,rdwinfo,<class 'dict'>,55
2017-04,rdwinfo,<class 'list'>,1
2017-05,rdwinfo,<class 'list'>,1
2017-06,rdwinfo,<class 'dict'>,53
2017-07,rdwinfo,<class 'dict'>,38
2017-08,rdwinfo,<class 'dict'>,45
2017-09,rdwinfo,<class 'dict'>,54
2017-10,rdwinfo,<class 'dict'>,46
2017-11,rdwinfo,<class 'dict'>,52
2017-12,rdwinfo,<class 'dict'>,54


In [10]:
# Other inconsistencies
if OPBOD:
    pass
else:
    # Other column names that changed in November 2017
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k] = data[k].rename(columns={
            'draw':'drz__Draw',
            'nr':'drz__LotNr',
            'raw':'drz__Raw_text',
            'misc':'drz__SupInfo',
            'footnote':'drz__Note',
            'jfq':'drz__jfc',
        })
        

In [11]:
for auct_date, df_orig in data.items():

    df = df_orig.copy()
    # standardize column names
    df.rename(columns=rename_dict, inplace=True)
    # with counters
    rn_dict = {}
    for k,v in rename_dict_with_counter.items():
        search_pat = re.sub('(_X)+(_\d+)?$', '_', k)
        sel_rn = df.columns.str.startswith(search_pat)
        if any(sel_rn):
            for col in df.columns[sel_rn]:
                if ('type1' in v) and ('type_1' in k):
                    counter = re.search('(_type_1)?((_\d+)+)$', col)[2] # ignore if ..type_1.. is in name
                else:
                    counter = re.search('(_\d+)+$', col)[0]
                newv = re.sub('(_X)+(_\d+)*$', counter + '\g<2>', v)
                rn_dict[col] = newv
                
                
    df.rename(columns=rn_dict, inplace=True)

    # this introduces duplicates in column names
    # drop second, keep first
    vc = df.columns.value_counts()
    duplicate_cols = vc[vc>1].index
    for c in duplicate_cols:
        # get column number
        idx = np.where(df.columns.get_loc(c))[0]
        assert len(idx) == 2 # for now only do first/second
        # update first col
        df.iloc[:,idx[0]] = df.iloc[:,idx[0]].combine_first(df.iloc[:,idx[1]])
        # drop second col
        df = df.iloc[:, [j for j in range(len(df.columns)) if j != idx[1]]]

    data[auct_date] = df
    del df_orig

# Merge all auctions into one data frame

In [12]:
all_data = pd.concat(data)
df = all_data.droplevel(0)

In [13]:
if VERBOSE > 0:
    # Show columns added
    col_exist = pd.DataFrame.from_dict({k:all_data.columns.isin(d.columns) for k,d in data.items()}, orient='index', columns=all_data.columns)
    # deprecated: col_filled = all_data.isna().all(level=0, axis=0) == False
    col_filled = all_data.notna().groupby(level=0).any()

    idx = col_exist.apply(lambda x:np.where(x)[0])

    first_occurence = col_exist.index[idx.apply(lambda x: x[0])]
    last_occurence = col_exist.index[idx.apply(lambda x: x[-1])]
    nr_occurence = idx.apply(len)
    pct_occurence = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])))

    idx = col_filled.apply(lambda x:np.where(x)[0])
    last_filled = col_filled.index[idx.apply(lambda x: x[-1] if len(x)>0 else col_exist.shape[0]-1)]
    nr_filled = idx.apply(len)
    pct_filled = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])) if len(x)>0 else 0)


    df_ = pd.DataFrame(zip(first_occurence, last_occurence, nr_occurence, last_filled, nr_filled, pct_occurence, pct_filled), 
                       index=col_exist.columns, 
                       columns=['first', 'last', 'n', 'last_filled', 'n_not_empty', 'pct_since_first', 'pct_filled'])



    for _by_source, df__ in df_.groupby(df_.index.str.split('_').str.get(0)):
        print(_by_source)
        for g,df___ in df__.groupby('first'):          
            if g == col_exist.index[0]:
                print(f'First data in {g}: {df___.shape[0]:.0f}')
                continue
            else:
                print(f'Fields added in {g}', end= ' ')
            with pd.option_context("display.max_rows", 9999):
                display(df___)#.sort_index())#.sort_values(by='last_filled', ascending=False))

drz
First data in 2014-10: 72
Fields added in 2017-11 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
drz__used_parts,2017-11,2017-11,1,2017-11,1,1.136364,1.136364


Fields added in 2018-01 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
drz__import22_btw21,2018-01,2018-07,2,2018-07,2,2.325581,2.325581
drz__import27_btw21,2018-01,2018-07,2,2018-07,2,2.325581,2.325581


Fields added in 2018-04 

...

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_3_7,2023-08-08,2023-10-10,3,2023-10-10,3,100.000000,100.000000
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_4_8,2023-08-08,2023-08-08,1,2023-08-08,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_3_7,2023-08-08,2023-10-10,3,2023-10-10,3,100.000000,100.000000
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_4_8,2023-08-08,2023-08-08,1,2023-08-08,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_soort_erkenning_keuringsinstantie_3_7,2023-08-08,2023-10-10,3,2023-10-10,3,100.000000,100.000000
rdw_geconstateerde_gebreken_met_beschrijving_soort_erkenning_keuringsinstantie_4_8,2023-08-08,2023-08-08,1,2023-08-08,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_3_7,2023-08-08,2023-10-10,3,2023-10-10,3,100.000000,100.000000
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_4_8,2023-08-08,2023-08-08,1,2023-08-08,1,33.333333,33.333333
rdw_geconstateerde_gebreken_met_beschrijving_meld_tijd_door_keuringsinstantie_3_7,2023-08-08,2023-10-10,3,2023-10-10,3,100.000000,100.000000
rdw_geconstateerde_gebreken_met_beschrijving_meld_tijd_door_keuringsinstantie_4_8,2023-08-08,2023-08-08,1,2023-08-08,1,33.333333,33.333333


Fields added in 2023-09-09 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_terugroep_actie_status_referentiecode_rdw_5,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0
rdw_terugroep_actie_status_referentiecode_rdw_6,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0
rdw_terugroep_actie_status_code_status_5,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0
rdw_terugroep_actie_status_code_status_6,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0
rdw_terugroep_actie_status_status_5,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0
rdw_terugroep_actie_status_status_6,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_4_7,2023-09-09,2023-10-10,2,2023-10-10,2,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_5_7,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_4_7,2023-09-09,2023-10-10,2,2023-10-10,2,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_5_7,2023-09-09,2023-09-09,1,2023-09-09,1,50.0,50.0


Fields added in 2023-10-10 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_bijzonderheden_bijzonderheid_volgnummer_2,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_bijzonderheden_bijzonderheid_code_2,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_bijzonderheden_bijzonderheid_code_1_2,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_3_10,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_1_11,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_2_10,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_meld_datum_door_keuringsinstantie_dt_2_12,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_3_10,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_1_11,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_2_10,2023-10-10,2023-10-10,1,2023-10-10,1,100.0,100.0


- - - -
# Continue with cars only

In [14]:
# select data
car=df[df.drz__LotType == 'Personenauto'].copy()

if VERBOSE > 0:
    # print lots that are cars per auction
    df_ = pd.DataFrame(columns=['year', 'month', 'lot'])
    df_.year = pd.Series(car.index).apply(lambda x: int(x.split('-')[0]))
    df_.month = pd.Series(car.index).apply(lambda x: int(x.split('-')[1]))
    df_.lot = pd.Series(car.index).apply(lambda x: int(x.split('-')[2]))
    with pd.option_context('display.max_rows', 999):
        display(df_.groupby(['year', 'month']).lot.agg(['min', 'max', 'count']).sort_index())
    

display(car.tail(), metadata={"tags":(TAG_SINGLE, )})

min     max  count
year month                       
2014 10       2000    7312    136
     11       2400    8198    133
     12       2000    7306    147
2015 1        2402    8318    134
     2        2000    7318    138
     3        2400    8312    157
2017 3        2000    8318    149
     4        2000    7309    133
     5        2200    8025     75
     6        2000    7311    140
     7        2200    8178    106
     8        2001    7304    143
     9        2200    8187    133
     10       2000    7171     95
     11       2200    8305    158
     12       2000    7308    136
2018 1        2200    8170    134
     2        2000    7184    138
     3        2200    8338    186
     4        2000    7336    126
     5        2200    8226    186
     6        2000    7315    166
     7        2200    8311    132
     8        2000    7328    152
     9        2200    8183     89
     10       2000    7319     87
     11       2200    8326    127
     12       2000    7361    133
2019 1        2201    8329    135
     2        2001    7264    176
     3        2200    8318    145
     4        2000    7314    170
     5        2200    8352    158
     6        2201    9158     98
     7        2200    7254    144
     8        2200    8172     96
     9        2200    9269    156
     10       2400    7200    109
     11       2200    8213    125
     12       2200    9061     75
2020 1        2200    7205    126
     2        2200    8238    129
     3        2200    9152     65
     6        2200    7269    160
     7        2200    9271    145
     8        2200    7286    187
     9        2200    8236    111
     10       2200    7266    129
     11       2200    8260    121
     12       2201    7306    168
2021 1        2200    8123    104
     2        2200    7156    129
     3        2200    8189    148
     4        2200    7220    173
     5        2200    8158    126
     6      220006  808316    191
     7      201217  900007    152
     8      260018  812918    143
     9      220009  808919    135
     10     260110  814920    181
     11     262411  805621     97
     12     260012  807722    171
2022 1      220021  805621    115
     2      240002  810122    193
     3      220003  807723    132
     4      240004  805724     88
     5      260025  805125    103
     6      700006  712506     79
     7      260007  982727    112
     8      260008  803228     81
     9      265029  803429    106
     10     700030  807010     67
     11     260011  803311    116
     12     260032  706912     65
2023 1      260001  709701     67
     2      700002  707202     48
     3      261603  712003     84
     4      200004  710404     87
     5      700005  707205     45
     6      260306  707706     66
     7      260007  810707     59
     8      702508  802308     40
     9      261909  710209     67
     10     262210  708910     56

# Save file

In [15]:
file_name = r'../data/cars-from-all-auctions.pkl'
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    car.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    


../data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [16]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [17]:
file_name = '../assets/example-table-of-cars.md'

example = car.tail(10).copy()

# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'

# convert to md
try:
    table_text = example.to_markdown()
except ImportError:
    print('Fallback')
    table_text = pandas_df_to_markdown_table(example)

# save
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    with open(file_name,'w') as file:
        file.write(table_text)

    print('A markdown table is available as\n\t{}'.format(file_name))
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    display(example, metadata={"tags":(TAG_SINGLE, )})
    


Skip. ../assets/example-table-of-cars.md exists or saving is disabled in settings.


In [18]:
# idx = df.index.str.split('-', expand=True).droplevel(-1)
# is_last_auction = idx == idx[-1]
# subsel_reg = df.loc[is_last_auction, 'Reg'].drop_duplicates().values
# VERBOSE

In [19]:
# vc2

In [20]:
# show recurring lots (based on registration no.)

vc = df.drz__Reg.value_counts()

# drop obvious
vc.drop(['onbekend', ''], inplace=True, errors='ignore')

# reduce output
if VERBOSE < 2:
    # only of last auction
    idx = df.index.str.split('-', expand=True).droplevel(-1)
    is_last_auction = idx == idx[-1]
    subsel_reg = df.loc[is_last_auction, 'drz__Reg'].drop_duplicates().values
    vc = vc[vc.index.isin(subsel_reg)]
    
# show recurring
for r in vc[vc > 1].index:
    # title
    print(r, vc[r], end=': ')
    sel_reg = df.drz__Reg==r
    print(' - '.join(list(df.loc[sel_reg, [
        'drz__ItemBrand', 'drz__ItemType', 'rdw_gekentekende_voertuigen_datum_eerste_tenaamstelling_in_nederland'
    ]].drop_duplicates(ignore_index=True).iloc[0].astype(str))), end=' ')
    
    df_ = df.loc[sel_reg, list(df.columns.drop([
        'drz__Raw_text', 'rdwinfo', 'rdw_merk_registratie_datum', 'rdw_merk_registratie_datum_dt', 'rdw_merkcode', 'drz__Images', 'drz__Source'
    ], errors='ignore'))].sort_values(by='drz__Reg').T

    # convert index to nr of days since first
    ix = pd.Series(df_.columns.map(lambda x: pd.to_datetime('/'.join(x.split('-')[:2])))).diff().dt.days.copy()
    ix[0] = 0
    ix=ix.astype(int).astype(str)
    ix[0] = pd.to_datetime('/'.join(df_.columns[0].split('-')[:2])).strftime('%Y/%m')

    df_.columns=ix

    if (VERBOSE < 2) | (df_.shape[1] > 4):

        vc2 = df_.applymap(lambda x: str(x) if isinstance(x, list) else x).nunique(axis=1)
        print('')
        with pd.option_context('display.max_rows', 999):
            display(df_.loc[list(vc2[vc2>1].index),:])
    elif df_.shape[1] > 2:
        display(df_.loc[['Price'],:].T)
    else:
        print(*list(df_.columns))
        

KS-15-58 3: KTM - 400 ls-e/mil - nan 


,2022/11,120,214
drz__Title,Kavel K2200112613,Kavel K2300032613,Kavel K2300102602
drz__Price,2287.0,4444.0,2506.0
drz__lot_counter,2613.0,2613.0,2602.0
drz__LotNr,K2200112613,K2300032613,K2300102602
rdw_ovi_TimeStamp,NaN,2023-02-11 16:51:54+00:00,2023-05-27 09:28:17+00:00


DPF-41-F 3: KRAL - grande5000 - 20180126.0 


,2023/01,151,122
drz__Title,Kavel K2300012601,Kavel K2300062602,Kavel K2300102620
drz__Price,1866.0,1111.0,0.0
drz__N_images,12.0,15.0,15.0
drz__lot_counter,2601.0,2602.0,2620.0
drz__LotNr,K2300012601,K2300062602,K2300102620
drz__SupInfo,\nLaadkabel aanwezig,\nOplaadkabel aanwezig\nDe elektromotor van de...,\nOplaadkabel aanwezig\nDe elektromotor van de...
rdw_gekentekende_voertuigen_datum_tenaamstelling,20221201.0,20230220.0,20230220.0
rdw_gekentekende_voertuigen_datum_tenaamstelling_dt,2022-12-01T00:00:00.000,2023-02-20 00:00:00,2023-02-20 00:00:00
rdw_gekentekende_voertuigen_datum_eerste_toelating_dt,2018-01-26T00:00:00.000,2018-01-26 00:00:00,2018-01-26 00:00:00
rdw_gekentekende_voertuigen_datum_eerste_tenaamstelling_in_nederland_dt,2018-01-26T00:00:00.000,2018-01-26 00:00:00,2018-01-26 00:00:00


KW-82-89 3: BURG - bpo 12-27 - nan 


...

,2023/06,122
drz__Title,Kavel K2300061904,Kavel K2300101928
drz__Price,500.0,457.0
drz__lot_counter,1904.0,1928.0
drz__LotNr,K2300061904,K2300101928
drz__no_orireg,True,False
rdw_gekentekende_voertuigen_TimeStamp,2023-03-25 09:25:00+00:00,2023-05-27 09:20:05+00:00
rdw_ovi_TimeStamp,2023-03-25 09:31:58+00:00,2023-05-27 09:28:17+00:00
rdw_brandstof_TimeStamp,2023-03-25 09:25:01+00:00,2023-05-27 09:20:09+00:00
rdw_eeg_voertuigtypegoedkeuring_TimeStamp,2023-03-25 09:25:14+00:00,2023-05-27 09:20:35+00:00
rdw_basisgegevens_eeg_uitvoering_TimeStamp,2023-03-25 09:25:30+00:00,2023-05-27 09:21:00+00:00


F-544-XD 2: SYM - xs50qt-7 - 20140721.0 


,2023/06,122
drz__Title,Kavel K2300061932,Kavel K2300101923
drz__Price,289.0,444.0
drz__N_images,7.0,8.0
drz__lot_counter,1932.0,1923.0
drz__LotNr,K2300061932,K2300101923
rdw_gekentekende_voertuigen_TimeStamp,2023-03-25 09:25:00+00:00,2023-05-27 09:20:05+00:00
rdw_ovi_TimeStamp,2023-03-25 09:31:58+00:00,2023-05-27 09:28:17+00:00
rdw_brandstof_TimeStamp,2023-03-25 09:25:01+00:00,2023-05-27 09:20:09+00:00
rdw_eeg_voertuigtypegoedkeuring_TimeStamp,2023-03-25 09:25:14+00:00,2023-05-27 09:20:35+00:00
rdw_basisgegevens_eeg_uitvoering_TimeStamp,2023-03-25 09:25:30+00:00,2023-05-27 09:21:00+00:00


DJB-98-V 2: PIAGGIO - vespa sprint - 20170105.0 


,2023/06,122
drz__Title,Kavel K2300061935,Kavel K2300101922
drz__Price,2000.0,955.0
drz__lot_counter,1935.0,1922.0
drz__LotNr,K2300061935,K2300101922
rdw_gekentekende_voertuigen_TimeStamp,2023-03-25 09:25:00+00:00,2023-05-27 09:20:05+00:00
rdw_ovi_TimeStamp,2023-03-25 09:31:58+00:00,2023-05-27 09:28:17+00:00
rdw_brandstof_TimeStamp,2023-03-25 09:25:01+00:00,2023-05-27 09:20:09+00:00
rdw_eeg_voertuigtypegoedkeuring_TimeStamp,2023-03-25 09:25:14+00:00,2023-05-27 09:20:35+00:00
rdw_basisgegevens_eeg_uitvoering_TimeStamp,2023-03-25 09:25:30+00:00,2023-05-27 09:21:00+00:00
rdw_handelsbenaming_uitvoering_TimeStamp,2023-03-25 09:25:45+00:00,2023-05-27 09:21:22+00:00
